   <center><h1>This is a dev and test environment for EMD1.0</h1></center>

In [ ]:
!pip install py4j
import py4j.java_gateway as jg
from py4j.java_gateway import JavaGateway
class NetLogo_EMD_Controller_Connector:
    gw = None    
    gw = JavaGateway()# New gateway connection 
    
    ###Internal method to start JavaGateway server###
    def __runServer(): 
        env = dict(os.environ)
        env['JAVA_OPTS'] = ''
        print(subprocess.call(["C:/Users/ch328575/Documents/Projects/EMD/EMD1.0/bin/startserver.bat"], env=env))
    ###Internal method to start JavaGateway server###
    def startServer():
        #Fire up the NetLogo Controller server through python
        thread = threading.Thread(target=__runServer, args=())
        thread.start()
    
    def run_model(model_string, steps, metric):
        bridge = gw.entry_point
        bridge.refresh()
        print(bridge.openModel(model_string))
        setParamsRandom(gw,bridge, model_string)
        bridge.command("setup")
        bridge.command("repeat " + str(steps) + " [go]")
        result = bridge.report(str(metric))
        bridge.closeModel()
        return result

    def disconnect():
        gw.shutdown(True)
    
        
    def setParamsRandom(gateway, bridge, model_string):
        paramSpecs = bridge.getParamList(model_string).getParamSpecs()

        for paramSpec in paramSpecs:
            if jg.is_instance_of(gateway,paramSpec,"bsearch.space.DoubleDiscreteSpec"):
                paramValue = paramSpec.generateRandomValue(gateway.jvm.org.nlogo.api.MersenneTwisterFast())
            if jg.is_instance_of(gateway,paramSpec,"bsearch.space.DoubleContinuousSpec"):
                paramValue = paramSpec.generateRandomValue(gateway.jvm.org.nlogo.api.MersenneTwisterFast())
            if jg.is_instance_of(gateway,paramSpec,"bsearch.space.CategoricalSpec"):
                paramValue = paramSpec.generateRandomValue(gateway.jvm.org.nlogo.api.MersenneTwisterFast())
                if type(paramValue) != bool:#isinstance(data[i][k], bool)
                    paramValue = '"{}"'.format(paramSpec.generateRandomValue(gateway.jvm.org.nlogo.api.MersenneTwisterFast()))
            if jg.is_instance_of(gateway,paramSpec,"bsearch.space.ConstantSpec"):
                paramValue = paramSpec.generateRandomValue(gateway.jvm.org.nlogo.api.MersenneTwisterFast())
            print("set " + str(paramSpec.getParameterName()) + " " + str(paramValue))
            bridge.command("set " + str(paramSpec.getParameterName()) + " " + str(paramValue))
        bridge.command("set " + str(paramSpec.getParameterName()) + " " + str(paramValue))
        bridge.command("set historic-view? true" )